In [ ]:
%pip install langchain
%pip install "langchain-community[pdf]"
%pip install faiss-cpu
%pip install sentence-transformers
%pip install transformers
%pip install accelerate

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os


print("Loading the saved vector index...")

# The name of the folder where you saved the index
index_folder_name = "my_vector_index"

# --- NEW ERROR HANDLING ---
# Check if the index folder exists before trying to load it.
if not os.path.exists(index_folder_name):
    print(f"---!!! ERROR !!!---")
    print(f"The vector index folder '{index_folder_name}' was not found.")
    print("Please make sure you have successfully run the 'build_index.ipynb' notebook first.")
    print("You should see a folder named 'my_vector_index' in your file explorer on the left.")
    # Stop the script if the index doesn't exist.
    raise FileNotFoundError(f"Vector index folder not found: {index_folder_name}")


# The same embedding model used to create the index
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Load the vector store from the local folder
vector_store = FAISS.load_local(index_folder_name, embeddings, allow_dangerous_deserialization=True)

print("Vector index loaded successfully!")


In [ ]:
# -----------------------------------------------------------------
# Part 2: Connect to a Free Language Model (LLM) from Hugging Face
# -----------------------------------------------------------------
# This new section loads a free, open-source model that runs on your
# compute instance, instead of using the paid Azure OpenAI service.

print("Loading free language model from Hugging Face... (This may take a few minutes for the first run)")

# --- MOST ADVANCED MODEL ---
# We are now using a state-of-the-art small language model for high-quality answers.
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Load the tokenizer and model. This model requires 'trust_remote_code=True'.
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto" # Automatically use GPU if available
)

# Create a text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512, # The number of tokens to generate for an answer
    return_full_text=False # We only want the generated answer
)

# Create a LangChain wrapper for the pipeline
llm = HuggingFacePipeline(pipeline=pipe)

print("Successfully loaded the free Language Model.")

In [ ]:
# Part 3: Create the Question-Answering Chain
# -----------------------------------------------------------------
# This chain combines the vector index (for retrieving information)
# and the LLM (for generating answers).
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="refine", # Changed to "refine" for more detailed answers
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
)

print("Question-Answering chain is ready.")


# -----------------------------------------------------------------
# Part 4: Ask Questions!
# -----------------------------------------------------------------
# Now you can ask questions about your documents in a loop.
# To exit, type 'quit' or 'exit'.

print("\n--- GATE DSAI Study Bot ---")
print("Enter your question. Type 'quit' to exit.")

while True:
    query = input("\nYour Question: ")
    if query.lower() in ["quit", "exit"]:
        print("Goodbye!")
        break
    if query.strip() == "":
        continue

    try:
        # Ask the chain a question
        result = qa_chain.invoke({"query": query})

        # Print the answer
        print("\nAnswer:")
        print(result["result"])

        # Print the sources it used
        print("\nSources:")
        for doc in result["source_documents"]:
            # We access the metadata to get the source file name
            print(f"- {doc.metadata.get('source', 'Unknown source')}")

    except Exception as e:
        print(f"An error occurred: {e}")
